In [ ]:
import PyPDF2 as pypdf

In [ ]:
pdfObj = open('../../data/raquel_documents/Pfizer/US9175093.pdf', 'rb')

In [ ]:
pdfReader = pypdf.PdfFileReader(pdfObj)

In [ ]:
firstPage = pdfReader.getPage(0)

In [ ]:
print(firstPage.extractText())

In [ ]:
from pdfminer.pdfparser import PDFParser, PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LAParams, LTTextBox, LTTextLine

In [ ]:
parser = PDFParser(pdfObj)

In [ ]:
doc = PDFDocument()

In [ ]:
parser.set_document(doc)

In [ ]:
doc.set_parser(parser)

In [ ]:
doc.initialize('')
rsrcmgr = PDFResourceManager()
laparams = LAParams()

In [ ]:
#laparams.char_margin = 1.0
#laparams.word_margin = 1.0
device = PDFPageAggregator(rsrcmgr, laparams=laparams)
interpreter = PDFPageInterpreter(rsrcmgr, device)
extracted_text = ''

In [ ]:
for page in doc.get_pages():
    interpreter.process_page(page)
    layout = device.get_result()
    for lt_obj in layout:
        if isinstance(lt_obj, LTTextBox) or isinstance(lt_obj, LTTextLine):
            extracted_text += lt_obj.get_text()

In [ ]:
raw_text = extracted_text.encode("utf-8")
print(raw_text)

In [ ]:
import re
print(re.sub(rb'\n', rb' ', raw_text))

In [ ]:
word_list = re.sub(rb'\n', rb' ', raw_text).split()

In [ ]:
epitope_i = [i for i,x in enumerate(word_list) if x==rb'epitope']

In [ ]:
len(epitope_i)

In [ ]:
word_list[(epitope_i[3] - 10):(epitope_i[3] + 100)]

In [ ]:
clean_text = re.sub(rb'\n', rb' ', raw_text)

In [ ]:
import nltk

In [ ]:
print(clean_text)

In [ ]:
#clean_text.encode("ascii","ignore")
udata=clean_text.decode("utf-8")
asciidata=udata.encode("ascii","ignore")

In [ ]:
print(udata)

In [ ]:
import sys
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.text import Text
tokens = word_tokenize(udata)
textList = Text(tokens)
textList.concordance('epitope', width=300, lines=50)
#print(tokens)


In [ ]:
textList.concordance('residues', width=300, lines=50)

In [ ]:
textList.similar('epitope')

In [ ]:
textList.similar('residue')

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [ ]:
sent_tokens = sent_tokenize(udata)
epitope_sent = []
for sent in sent_tokens:
    if 'epitope' in sent:
        epitope_sent.append(sent)
print(len(epitope_sent))

In [ ]:
epitope_sent[0]

In [ ]:
epitope_sent[1]

In [ ]:
epitope_seq_sent = []
for sent in epitope_sent:
    if 'SEQ' in sent:
        epitope_seq_sent.append(sent)
print(len(epitope_seq_sent))

In [ ]:
epitope_seq_sent[0]

In [ ]:
epitope_seq_sent[1]

In [ ]:
doc = nlp(epitope_seq_sent[0])

In [ ]:
for ent in doc.ents:
    print(ent.text, ent.label_)

In [ ]:
spacy.explain("ORG")

In [ ]:
spacy.explain("CARDINAL")

In [5]:
def extract_text(path):
    with open(path, 'rb') as pdf_obj:
        parser = PDFParser(pdf_obj)
        doc = PDFDocument()
        parser.set_document(doc)
        doc.set_parser(parser)

        doc.initialize('')
        rsrcmgr = PDFResourceManager()
        laparams = LAParams()

        #laparams.char_margin = 1.0
        #laparams.word_margin = 1.0
        device = PDFPageAggregator(rsrcmgr, laparams=laparams)
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        extracted_text = ''

        for page in doc.get_pages():
            interpreter.process_page(page)
            layout = device.get_result()
            for lt_obj in layout:
                if isinstance(lt_obj, LTTextBox) or isinstance(lt_obj, LTTextLine):
                    extracted_text += lt_obj.get_text()

        raw_text = extracted_text.encode("utf-8")
        clean_text = re.sub(rb'\n', rb' ', raw_text)
        udata = clean_text.decode("utf-8")

        return udata

def get_word_list(path):
    return word_tokenize(extract_text(path))

def get_sentence_list(path):
    return sent_tokenize(extract_text(path))

def extract_epitope_seq_sentences(path):
    sent_tokens = get_sentence_list(path)
    epitope_seq_sent = []
    for sent in sent_tokens:
        if 'SEQ' in sent and 'epitope' in sent:
            epitope_seq_sent.append(sent)
    return epitope_seq_sent

def extract_entities(sentence):
    doc = nlp(sentence)
    for ent in doc.ents:
        print(ent.text, ent.label_)
base_path = '/Users/parvinderjit.singh/Desktop/Harvard/CSCI E-599/data/raquel_documents/'
add_path = 'Pfizer/US9175093.pdf'


In [6]:
from pdfminer.pdfparser import PDFParser, PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LAParams, LTTextBox, LTTextLine
import re
from nltk.tokenize import sent_tokenize, word_tokenize
import spacy

In [7]:
my_sent = extract_epitope_seq_sentences(base_path + add_path)

In [10]:
nlp = spacy.load('en_core_web_sm')

In [11]:
for sent in my_sent:
    extract_entities(sent)

PCSK9 ORG
PCSK9 ORG
153-155 DATE
194 CARDINAL
195 CARDINAL
197 CARDINAL
237 CARDINAL
367 CARDINAL
369 CARDINAL
374-379 CARDINAL
381 CARDINAL
PCSK9 ORG
SEQ ORG
188 CARDINAL
PCSK9 ORG
153-155 CARDINAL
194 CARDINAL
197 CARDINAL
237 CARDINAL
367 CARDINAL
SEQ ORG
NO:188 ORG


In [12]:
my_sent = extract_epitope_seq_sentences(base_path + 'Amgen/us8829165.pdf')
for sent in my_sent:
    extract_entities(sent)

PCSK9 ORG
CDRH ORG
at least one CARDINAL
CDRH3 ORG
CDRH3 ORG
SEQ ORG
67 CARDINAL
79 DATE
49 DATE
CDRH3 ORG
more than two CARDINAL
111 CARDINAL
X3X4X5X NORP
Xi3Xi4 PERSON
SEQ ORG
404 CARDINAL
X~ PERSON
X3 ORG
X4 ORG
X5 GPE
W GPE
X~ PERSON
X~ PERSON
at least one CARDINAL
CDRL3 ORG
CDRL3 ORG
SEQ ORG
12 CARDINAL
35 DATE
23 CARDINAL
CDRL3 ORG
405 CARDINAL
more than two CARDINAL
CDRL3 ORG
X7XsXgXiOXi PERSON
SEQ ORG
X~ PERSON
X3 ORG
X4 ORG
X5 GPE
X~ PERSON
L GPE
S GPE
X~ PERSON
5 CARDINAL
7 DATE
9 DATE
10 DATE
12 DATE
13 DATE
15 DATE
16 DATE
17 DATE
18 DATE
19 DATE
20 DATE
21 DATE
22 DATE
23 DATE
24 DATE
26 DATE
28 DATE
30 DATE
31 DATE
32 DATE
33 DATE
35 DATE
36 DATE
37 DATE
38 DATE
39 DATE
40 DATE
42 DATE
44 DATE
46 DATE
at least one CARDINAL
31 CARDINAL
SEQ ORG
3 CARDINAL
PCSK9 ORG
SEQ ORG
3 CARDINAL
at least one CARDINAL
SEQ ORG
207 CARDINAL
208 CARDINAL
181 CARDINAL
185 CARDINAL
439 CARDINAL
513 CARDINAL
538 CARDINAL
539 CARDINAL
132 CARDINAL
351 CARDINAL
390 CARDINAL
413 CARDINAL
582 CAR

In [14]:
len(my_sent)

8

In [28]:
extract_entities(my_sent[2])

PCSK9 ORG
SEQ ORG
3 CARDINAL
at least one CARDINAL
SEQ ORG
207 CARDINAL
208 CARDINAL
181 CARDINAL
185 CARDINAL
439 CARDINAL
513 CARDINAL
538 CARDINAL
539 CARDINAL
132 CARDINAL
351 CARDINAL
390 CARDINAL
413 CARDINAL
582 CARDINAL
162 CARDINAL
164 CARDINAL
167 CARDINAL
123 CARDINAL
129 CARDINAL
311 CARDINAL
313 CARDINAL
337 CARDINAL
519 CARDINAL
521 CARDINAL
554 CARDINAL


In [39]:
my_sent = extract_epitope_seq_sentences(base_path + 'Regeneron/US2010166768.pdf')

In [40]:
my_sent

['In one embodiment,  the antibody or antigen-binding  is characterized as binding an epitope comprising  [0015]  [0016]  fragment  amino acid residue 238 of hPCSK9 (SEQ ID NO:755).',
 'In a  more specific embodiment,  the antibody or antigen-binding  fragment binds an epitope comprising one or more of amino  acid residue 238, 153, 159 and 343 of hPCSK9 (SEQ ID  NO:755).',
 'In a more specific embodiment,  the antibody or  fragment thereof is characterized as binding an epitope which  does not comprise an amino acid residue at position 192, 194,  197 and/or 237 of SEQ ID NO:755.',
 '[0017]  In one embodiment,  the antibody or antigen-binding  is characterized as binding an epitope comprising  fragment  amino acid residue 366 of hPCSK9 (SEQ ID NO:755).',
 'In a  more specific embodiment,  the antibody or antigen-binding  fragment binds an epitope comprising one or more of amino  acid residue at position 147, 366 and 380 of SEQ ID NO:755.',
 'In a more specific embodiment,  the antibody 

In [41]:
len(my_sent)

16

In [42]:
sentences = get_sentence_list(base_path + add_path)

In [127]:
import re
import os
from collections import Counter 
import logging

pat_num_pattern = "US\s\d\d?,\d\d\d,\d\d\d\s[A-B]\d"
pat_num_pattern2 = "US\s\d\d\d\d/\d*\s[A-B]\d"
pat_id_pattern = "US\s?\d\d\d\d\d\d\d\d\d*[A-B][I0-9]"
pat_date_match = "[JFMASOND][aepuco][nbyrlgptvc]\.?\s\d\d?,\s\d\d\d\d"
pat_num_matcher = re.compile(pat_num_pattern)
pat_num_matcher2 = re.compile(pat_num_pattern2)
pat_id_matcher = re.compile(pat_id_pattern)
pat_date_matcher = re.compile(pat_date_match)
logging.getLogger('pdfminer').setLevel(logging.ERROR)


In [149]:
patents = []
for root, directories, files in os.walk(base_path, topdown=False):
    for name in files:
        if '.pdf' in name:
            patents.append(os.path.join(root, name))
num_files = len(patents)
curr_file = 1
for patent in patents:
    print("Processing", curr_file, "of", num_files, ":", patent)
    sentences = get_sentence_list(patent)
    patent_nums = []
    patent_ids = []
    sentences[0] = sentences[0].replace(")", "1")
    date_matched = False
    concat_sent = ''
    pat_date = None
    for s in sentences[0:20]:
        concat_sent += s
    pat_date_match = pat_date_matcher.search(concat_sent)
    if pat_date_match:
        pat_date = pat_date_match.group(0)
    for sent in sentences:
        pat_num_matches = pat_num_matcher.findall(sent)
        pat_num_matches += pat_num_matcher2.findall(sent)
        pat_id_matches = pat_id_matcher.findall(sent)
        
#        if not date_matched:
#            pat_date_match = pat_date_matcher.search(sent)
#            if pat_date_match: 
#                pat_date = pat_date_match.group(0)
#                date_matched = True
        
        if pat_num_matches: 
            for match in pat_num_matches:
                patent_nums.append(match)
        if pat_id_matches:
            for match in pat_id_matches:
                match = match.replace(" ", "")
                if match[-1] == "I": match = match[:-1] + '1'
                patent_ids.append(match)
                
    if len(patent_nums) > 0:
        pat_num_count = Counter(patent_nums)
        print(pat_num_count.most_common(1)[0][0])
    if len(patent_ids) > 0:
        pat_id_count = Counter(patent_ids)
        print(pat_id_count.most_common(1)[0][0])
    print(pat_date)
        
    print("\n===================================================================================\n")
    curr_file += 1

Processing 1 of 19 : /Users/parvinderjit.singh/Desktop/Harvard/CSCI E-599/data/raquel_documents/Genentech/us2012195910 claim 45.pdf
US 2012/0195910 A1
US20120195910A1
Aug. 2, 2012


Processing 2 of 19 : /Users/parvinderjit.singh/Desktop/Harvard/CSCI E-599/data/raquel_documents/Merck/us2009246192 epitopes at [0054].pdf
US 2009/0246192 A1
US20090246192A1
Oct. 1, 2009


Processing 3 of 19 : /Users/parvinderjit.singh/Desktop/Harvard/CSCI E-599/data/raquel_documents/Merck/us8188234 col14 ln40 epitope.pdf
US 8,188,234 B2
US008188234B2
May 29, 2012


Processing 4 of 19 : /Users/parvinderjit.singh/Desktop/Harvard/CSCI E-599/data/raquel_documents/Regeneron/US8501184.pdf
US 8,501,184 B2
US008501184B2
Aug. 6, 2013


Processing 5 of 19 : /Users/parvinderjit.singh/Desktop/Harvard/CSCI E-599/data/raquel_documents/Regeneron/US8357371.pdf
US 8,357,371 B2
US008357371B2
Jan. 22, 2013


Processing 6 of 19 : /Users/parvinderjit.singh/Desktop/Harvard/CSCI E-599/data/raquel_documents/Regeneron/US10023654 ep

US 2010/0166768 A1
US20100166768A1
Dec. 15, 2009


Processing 8 of 19 : /Users/parvinderjit.singh/Desktop/Harvard/CSCI E-599/data/raquel_documents/Regeneron/US9724411.pdf
US 9,724,411 B2
US009724411B2
Aug. 8, 2017


Processing 9 of 19 : /Users/parvinderjit.singh/Desktop/Harvard/CSCI E-599/data/raquel_documents/Regeneron/US8062640.pdf
US 8,062,640 B2
US008062640B2
Dec. 15, 2009


Processing 10 of 19 : /Users/parvinderjit.singh/Desktop/Harvard/CSCI E-599/data/raquel_documents/Regeneron/US9550837.pdf
US 9,550,837 B2
US009550837B2
Jan. 24, 2017


Processing 11 of 19 : /Users/parvinderjit.singh/Desktop/Harvard/CSCI E-599/data/raquel_documents/Amgen/us8563698 aa123-132.pdf
US 8,563,698 B2
US008563698B2
Oct. 22, 2013


Processing 12 of 19 : /Users/parvinderjit.singh/Desktop/Harvard/CSCI E-599/data/raquel_documents/Amgen/us8829165.pdf
US 8,829,165 B2
US008829165B2
Sep. 9, 2014


Processing 13 of 19 : /Users/parvinderjit.singh/Desktop/Harvard/CSCI E-599/data/raquel_documents/Amgen/us2009326202 

US 2009/0326202 A1
US20090326202A1
Dec. 31, 2009


Processing 14 of 19 : /Users/parvinderjit.singh/Desktop/Harvard/CSCI E-599/data/raquel_documents/Amgen/us8859741.pdf


KeyboardInterrupt: 

In [147]:
path = "/Users/parvinderjit.singh/Desktop/Harvard/CSCI E-599/data/raquel_documents/Genentech/us2012195910 claim 45.pdf"
sentences = get_sentence_list(path)
for sent in sentences:
    match = pat_date_matcher.search(sent)
    if match: 
        print("loop")
        print(match.group(0))
        break

loop
Aug. 2, 2012


In [148]:
concat_sent = ''
for s in sentences[0:20]:
    concat_sent += s 
print(pat_date_matcher.search(concat_sent).group(0))

Aug. 2, 2012


In [160]:
sentences[0]

'llIIlIlllIlIlIllIlIllIllIllIIlllIIllIlIlllIIlIIllllIllIlIIlllIllIllIIllIllIIIIIIIIIIIIIIII  US 20090326202AI  (191 United States  (121 Patent Application Publication  Jackson et al.'

In [141]:
concat_sent = ''
for s in sentences[0:10]:
    concat_sent += s 
print(concat_sent)

IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII US010023654B2 (12) United States Patent Sleeman et al.(10) Patent No.: US 1030233654 B2 (45) Date of Patent: *Jnl.17, 2018 (54) ANTI-PCSK9 ANTRIODIES (71) Appltcant: REOENERON PIIARMACEUTI()AI.S, IN(:., Tarrytown, NY (US) (72) Invmttors: Mark W. Sieeman7 Vfctona (AU)i Joel U. Martin.Putnam Valley.NY (US), Tammy T. Iluang.Cross River, NY (US); Douglas MacDonaid.New York, NY (US) (73) Assignee Regcneron Pharmaceuticals, Inc., Tarrytown, NY (US) (56) References Cited U.S. PATENT DOCUMENTS 5,260,440 4 5.27.",995 A I I 1993 HHSI 1211993 Roti (Continued) (52) U.S. CI.('P('.
